In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
import os

In [6]:
parent_folder = 'Data'

path = os.path.join(parent_folder, 'DATA_JUNAEB.pkl')

with open(path, 'rb') as f:
    data = pickle.load(f)


In [48]:
# Delete extra spaces in column 'Nivel Educativo'

for key in data:
    data[key]['Nivel Educativo'] = data[key]['Nivel Educativo'].str.strip()
    data[key]['Servicio'] = data[key]['Servicio'].str.strip()
    

In [60]:
path = os.path.join(parent_folder, 'Colegios(Continental)2020.xlsx')

colegios = pd.read_excel(path)

rbds = colegios['RBD'].values


In [83]:
def get_df_prices(data, rbds):
    prices = {}
    prices['RBD'] = list(rbds)

    for month in data:
        prices[month] = []
        for rbd in rbds:
            raciones = data[month][data[month]['Rbd'] == rbd]['Rac. Diaria Asig.']
            precio = data[month][data[month]['Rbd'] == rbd]['Precio Base']
            dias = data[month][data[month]['Rbd'] == rbd]['Dias ASI']
            
            total = raciones * precio * dias

            prices[month].append(total.sum())

    return pd.DataFrame(prices)

In [84]:
data_noCAN = {key: df[df['Servicio'] != 'CAN'] for key, df in data.items()}
data_CAN = {key: df[df['Servicio'] == 'CAN'] for key, df in data.items()}

prices_no_CAN = get_df_prices(data_noCAN, rbds)
prices_CAN = get_df_prices(data_CAN, rbds)

In [86]:
parent_folder = 'Data'
folder = 'Precios'

path1 = os.path.join(parent_folder, folder, 'Precios_noCAN.xlsx')
path2 = os.path.join(parent_folder, folder, 'Precios_CAN.xlsx')

prices_no_CAN.to_excel(path1, index=False)
prices_CAN.to_excel(path2, index=False)